In [ ]:
#| default_exp vision_plot

In [ ]:
#| export 
from fastai.data.all import *
from torchio.visualization import rotate
import cv2

# Vision plot
>

In [ ]:
#| export
def _get_slice(image, channel: int, indices: (int, list), anatomical_plane: int, voxel_size: (int, list)):
    """A private method to get a 2D tensor and aspect ratio for plotting.
    This is modified code from the torchio function `plot_volume`.

    Args:
        image: The input image.
        channel: Channel of the image.
        indices: Index of the 2D slice.
        anatomical_plane: Anatomical plane of the image.
        voxel_size: Voxel size for the image.

    Returns:
        A 2D tensor containing the sliced image and its aspect.
    """
    if voxel_size is None:
        warnings.warn('Voxel size not defined. Aspect ratio of the plot might not be correct.')
        voxel_size = 1

    if not is_listy(voxel_size):
        voxel_size = (voxel_size, voxel_size, voxel_size)

    data = image.data[channel]
    sr, sa, ss = voxel_size[0], voxel_size[1], voxel_size[2]
    sliced_img, aspect = None, None

    if indices is None:
        indices = np.array(data.shape) // 2

    i, j, k = indices

    if anatomical_plane == 1:
        sliced_img = rotate(data[:, j, :], radiological=True, n=1)
        aspect = ss / sr
    elif anatomical_plane == 2:
        sliced_img = rotate(data[:, :, k], radiological=True, n=1)
        aspect = sa / sr
    else:
        sliced_img = rotate(data[i, :, :], radiological=True, n=1)
        aspect = ss / sa

    return sliced_img, aspect

In [ ]:
#| export
@delegates(plt.Axes.imshow, keep=True, but=['shape', 'imlim'])
def show_med_img(im, ctx, channel: int, indices: (int, list), anatomical_plane: int,
                 voxel_size: (int, list), ax=None, figsize=None, title=None, **kwargs):
    """Show an image on `ax`. This is a modified code from the fastai function `show_image`.

    Args:
        im: The input image.
        ctx: The context.
        channel: Channel of the image.
        indices: Index of the 2D slice.
        anatomical_plane: Anatomical plane of the image.
        voxel_size: Voxel size for the image.
        ax: Axis for the plot.
        figsize: Figure size for the plot.
        title: Title for the plot.
        kwargs: Additional parameters for plt.Axes.imshow method.

    Returns:
        Axis with the plot.
    """
                     
    if hasattrs(im, ('data', 'cpu', 'permute')): # Check if `im` has the necessary attributes
        im = im.data.cpu()
        im, aspect = _get_slice(
            im, 
            channel=channel, 
            anatomical_plane=anatomical_plane,
            voxel_size=voxel_size, 
            indices=indices
        )

    ax = ax if ax is not None else ctx 

    if ax is None: # ax is only None when .show() is used.
        _, ax = plt.subplots(figsize=figsize)

    ax.imshow(im, aspect=aspect, **kwargs)

    if title is not None:
        ax.set_title(title)

    ax.axis('off')

    return ax

In [ ]:
#| export
def get_mid_slice(img, mask_data):
    """Extract a slice from the 3D image and mask data based on mask data.

    Args:
        img: [W, H, D] array.
        mask_data: [W, H, D] array.

    Returns:
        2D image and mask slice.
    """
    
    sums = mask_data.sum(axis=(0, 1))
    mid_idx = np.argmax(sums)
    img, mask_data = img[:, :, mid_idx], mask_data[:, :, mid_idx]

    return np.fliplr(np.rot90(img, -1)), np.fliplr(np.rot90(mask_data, -1))

In [ ]:
#| export
def get_fused_image(img, pred_mask, alpha=0.8):
    """Fuse a grayscale image with a mask overlay.

    Args:
        img: 2D grayscale image.
        pred_mask: 2D mask data.
        alpha: Weight of the grayscale image. Defaults to 0.8.

    Returns:
        A fused 2D array.
    """
    
    gray_img_colored = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    mask_color = np.array([0, 0, 255])
    colored_mask = (pred_mask[..., None] * mask_color).astype(np.uint8)

    return cv2.addWeighted(gray_img_colored, alpha, colored_mask, 1 - alpha, 0)